# **Data extraction and processing**

### To view the accesses allowed for web-scraping
[Acces for web scraping (titanicfacts.net by 2024 Dave Fowler) ](https://titanicfacts.net/robots.txt)

Install the libraries:

In [1]:
# %pip install pandas numpy unidecode

In [2]:
import pandas as pd
import re
import numpy as np
from unidecode import unidecode
pd.options.mode.copy_on_write = True 
URL = 'https://titanicfacts.net/titanic-passenger-list/'
dfs = pd.read_html(URL)

In [3]:
len(dfs)

3

### Columns of interest with their descriptions.

| **Column** | **Description**                                           |
| ---------- | --------------------------------------------------------- |
| Survived   | person survived the titanic tragedy (0 - "NO", 1- "YES" ) |
| Sex        | the gender (female, male)                                 |
| Name       | the name                                                  |
| Age        | Age in years                                              |
| SibSp      | # of siblings/wife or husband on board                    |
| Parch      | # of parents / children on board                          |
| Pclass     | The ticket class (1 = "1st", 2 = "2nd", 3 = "3rd")        |

In [4]:
dfs[0].head()

,0,1,2,3,4
0,Surname,First Names,Age,Boarded,Survivor (S) or Victim (†)
1,Allen,Miss Elisabeth Walton,29,Southampton,S
2,Allison,Mr Hudson Joshua Creighton,30,Southampton,†
3,Allison,Mrs Bessie Waldo,25,Southampton,†
4,Allison,Miss Helen Loraine,2,Southampton,†


### Data extraction from the web site

In [5]:
FIRST_ROW_OF_DF=0
TICKET_CLASS_COLUMN="Pclass"
SELECTED_COLUMNS_DEL_SITE_WEB_COLUMNS=["Surname", "First Names", "Age", "Pclass"]


def set_columns_of_row(df,row):
    new_columns=df.iloc[row]
    df.columns=new_columns

def delete_row(df,row_to_drop):
    df.drop(row_to_drop,inplace=True)

def add_columns_class_ticket(df,ticket_class):
    df[TICKET_CLASS_COLUMN]=ticket_class


def get_dataframe(df,ticket_class):
    set_columns_of_row(df,FIRST_ROW_OF_DF)
    delete_row(df,FIRST_ROW_OF_DF)
    add_columns_class_ticket(df,ticket_class)
    return df[SELECTED_COLUMNS_DEL_SITE_WEB_COLUMNS]


In [6]:
dfs=[get_dataframe(df,ticket_class+1) for ticket_class,df in enumerate(dfs)]
website_df=pd.concat(dfs)
website_df.reset_index(inplace=True,drop=True)


In [7]:
website_df.head()

,Surname,First Names,Age,Pclass
0,Allen,Miss Elisabeth Walton,29,1
1,Allison,Mr Hudson Joshua Creighton,30,1
2,Allison,Mrs Bessie Waldo,25,1
3,Allison,Miss Helen Loraine,2,1
4,Allison,Master Hudson Trevor,11m,1


In [8]:
website_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1317 entries, 0 to 1316
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Surname      1317 non-null   object
 1   First Names  1317 non-null   object
 2   Age          1317 non-null   object
 3   Pclass       1317 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 41.3+ KB


### Reading csv data

In [10]:
csv_df=pd.read_csv("titanic_final_project.csv")

In [11]:
csv_df.head()

,Survived,Name,Sex,Age,SibSp,Parch,Embarked
0,0,"Braund, Mr. Owen Harris",male,22.0,1,0,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
2,1,"Heikkinen, Miss. Laina",female,26.0,0,0,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
4,0,"Allen, Mr. William Henry",male,35.0,0,0,S


In [12]:
csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 48.9+ KB


Matching techniques:
1. Matching by surname and age, contemplating that there are no persons with the same surname and age.
2. Matching by full name using a set technique.
3. Match special cases manually.

### Data cleaning

The last name is separated from the first name of the passenger in the csv_df.

In [13]:
def extract_surname_and_first_name(full_name):
    separated_first_name = full_name.split(",")
    last_name = separated_first_name[0].strip().lower()
    first_name = separated_first_name[1].strip().lower()
    return last_name, first_name

def get_last_name_and_first_name(row):
    last_name, first_name = extract_surname_and_first_name(row["Name"])
    row["Surname"] = last_name
    row["Name"] = first_name
    return row

csv_df=csv_df.apply(get_last_name_and_first_name,axis='columns')

In [14]:
csv_df.head() 

,Survived,Name,Sex,Age,SibSp,Parch,Embarked,Surname
0,0,mr. owen harris,male,22.0,1,0,S,braund
1,1,mrs. john bradley (florence briggs thayer),female,38.0,1,0,C,cumings
2,1,miss. laina,female,26.0,0,0,S,heikkinen
3,1,mrs. jacques heath (lily may peel),female,35.0,1,0,S,futrelle
4,0,mr. william henry,male,35.0,0,0,S,allen


There are people with surnames and names with special characters in the website dataframe.

In [15]:
website_df[website_df["Surname"]=="Alhomäki"]

,Surname,First Names,Age,Pclass
623,Alhomäki,Mr Ilmari Rudolf,19,3


In [16]:
website_df[website_df["First Names"]=="Mr Far īd Husayn Qāsim"]

,Surname,First Names,Age,Pclass
621,Al-Munà,Mr Far īd Husayn Qāsim,18,3


Special characters are decoded.

In [17]:
website_df["Surname"]=website_df["Surname"].str.strip().apply(unidecode)
website_df["First Names"]=website_df["First Names"].str.strip().apply(unidecode)

The name is changed from uppercase to lowercase.

In [18]:
website_df["Surname"]=website_df["Surname"].str.lower()
website_df["First Names"]=website_df["First Names"].str.lower()

There are people with age in months and with type string in the variable age in the dataframe of the website.

In [19]:
def do_not_have_age_information(age):
    return age==""

def get_age_with_two_decimals(age):
    return round(age,2)

def age_is_in_months(age):
    return len(age)>1

def convert_age_to_years(row):
    age=re.split(r"[A-Za-z]",row["Age"])
    age_in_years=age[0]
    if do_not_have_age_information(age_in_years):
        row["Age"]=np.nan
    else:
        age_in_years = float(age_in_years)
        if age_is_in_months(age):
            age_in_years /= 12
        row["Age"] = get_age_with_two_decimals(age_in_years)
    return row

In [20]:
website_df=website_df.apply(convert_age_to_years,axis="columns")

Passengers that have the age attribute are separated, both in the CSV df and in the website df.

In [21]:
def has_age(df):
 return df["Age"].notna()

data_with_ages_not_null_csv=csv_df[has_age(csv_df)]
data_with_ages_not_null_web=website_df[has_age(website_df)]

In [22]:
data_with_ages_not_null_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Name      714 non-null    object 
 2   Sex       714 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     714 non-null    int64  
 5   Parch     714 non-null    int64  
 6   Embarked  712 non-null    object 
 7   Surname   714 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 50.2+ KB


In [23]:
data_with_ages_not_null_web.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1315 entries, 0 to 1316
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Surname      1315 non-null   object 
 1   First Names  1315 non-null   object 
 2   Age          1315 non-null   float64
 3   Pclass       1315 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 51.4+ KB


There are people with the same last name and the same age.

In [24]:
def has_the_same_last_name_and_age(df):
 return df[["Surname","Age"]].duplicated(keep=False)

In [25]:
data_with_ages_not_null_web[
    has_the_same_last_name_and_age(data_with_ages_not_null_web)
].sort_values(by="Age").head()

,Surname,First Names,Age,Pclass
650,asplund,master carl edgar,5.0,3
653,asplund,miss lillian gertrud,5.0,3
705,calic,mr jovo,17.0,3
704,calic,mr petar,17.0,3
197,marvin,mr daniel warner,18.0,1


In [26]:
data_with_ages_not_null_csv[
    has_the_same_last_name_and_age(data_with_ages_not_null_csv)
].sort_values(by="Age").head()

,Survived,Name,Sex,Age,SibSp,Parch,Embarked,Surname
469,1,miss. helene barbara,female,0.75,2,1,C,baclini
644,1,miss. eugenie,female,0.75,2,1,C,baclini
163,0,mr. jovo,male,17.00,0,0,S,calic
500,0,mr. petar,male,17.00,0,0,S,calic
404,0,miss. marija,female,20.00,0,0,S,oreskovic


Separamos a los que no tienen el mismo apellido y la misma edad.

In [27]:
data_with_ages_not_null_csv=data_with_ages_not_null_csv[
    ~has_the_same_last_name_and_age(data_with_ages_not_null_csv)]
data_with_ages_not_null_web=data_with_ages_not_null_web[
    ~has_the_same_last_name_and_age(data_with_ages_not_null_web)]

In [28]:
data_with_ages_not_null_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  700 non-null    int64  
 1   Name      700 non-null    object 
 2   Sex       700 non-null    object 
 3   Age       700 non-null    float64
 4   SibSp     700 non-null    int64  
 5   Parch     700 non-null    int64  
 6   Embarked  698 non-null    object 
 7   Surname   700 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 49.2+ KB


In [29]:
data_with_ages_not_null_web.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1281 entries, 0 to 1316
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Surname      1281 non-null   object 
 1   First Names  1281 non-null   object 
 2   Age          1281 non-null   float64
 3   Pclass       1281 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 50.0+ KB


### First way of matching

We do the inner join of the df of the web page with the df of the csv by the key (Last Name, Age).

In [30]:
match_first_form=pd.merge(data_with_ages_not_null_web
                          ,data_with_ages_not_null_csv
                          ,on=["Surname","Age"]
                          ,how="inner")

In [31]:
match_first_form.head()

,Surname,First Names,Age,Pclass,Survived,Name,Sex,SibSp,Parch,Embarked
0,allen,miss elisabeth walton,29.00,1,1,miss. elisabeth walton,female,0,0,S
1,allison,mrs bessie waldo,25.00,1,0,mrs. hudson j c (bessie waldo daniels),female,1,2,S
2,allison,miss helen loraine,2.00,1,0,miss. helen loraine,female,1,2,S
3,allison,master hudson trevor,0.92,1,1,master. hudson trevor,male,1,2,S
4,andrews,mr thomas,39.00,1,0,mr. thomas jr,male,0,0,S


In [32]:
match_first_form.reset_index(inplace=True,drop=True)

In [33]:
match_first_form.shape

(481, 10)